In [1]:
import dash 
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd 
import plotly.graph_objects as go 
import sys 

sys.path.append('../')

import omdutils as omd

In [32]:
df = omd.from_sqlite('SELECT * FROM cars', '../SourceData/datasets.db')
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [48]:
df = omd.from_sqlite('SELECT * FROM cars', '../SourceData/datasets.db')

app = dash.Dash(__name__)

origin = {1: 'American', 2: 'European', 3: 'Japanese'}
features = df.columns.to_list()
features = [feature for feature in df.columns if feature not in ['origin', 'name']]

app.layout = html.Div([
    html.H1('Car Dashboard'),
    html.Div([
        html.Label('Choose first feature (X-axis)'),
        dcc.Dropdown(
            id='feature-1',
            options=[{'label': i, 'value': i} for i in features],
            value='mpg'  
        )
    ], style={'width': '48%', 'display':'inline-block'}),
    html.Div([
        html.Label('Choose second feature (Y-axis)'),
        dcc.Dropdown(
            id='feature-2',
            options=[{'label': i, 'value': i} for i in features],
            value='weight'  
        )
    ], style={'width': '48%', 'display':'inline-block'}),
    html.Div([
        dcc.Graph(id='graph')
    ]),
], style={'padding':10})

@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('feature-1', 'value'),
     dash.dependencies.Input('feature-2', 'value')]
)
def update_graph(feature1, feature2):
    fig = px.scatter(
        df, 
        x=feature1, 
        y=feature2, 
        color=df['origin'].map(origin),  
        color_discrete_map={
            'American': 'orange',
            'European': 'blue',
            'Japanese': 'green'
        },
        height=600,  
        size='cylinders',  
        title=f'{feature1} vs {feature2}',
        labels={'cylinders': 'Cylinders'},  
        hover_data={'name': True, 'model_year': True}
    )
    fig.update_layout(
        title={
            'text': f'{feature1.upper()} vs {feature2.upper()}',
            'x': 0.5,  # Center the title
            'xanchor': 'center'
        },
        hovermode='closest'
    )
    return fig

app.run(debug=True)